In [4]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

from scipy.optimize import fmin

from sympy import *

In [ ]:
# O esquemático básico é:

pop = init_pop_rand
avalia_pop(pop)
while condicao:
    t += 1
    selec_pai(pop)
    crossover
    mutacao
    avalia_pop(pop)

In [230]:
def funcao_a_minimizar(x):
    return x**2 + 0.3*np.cos(10*np.pi*x)

def init_indv_rand(num_bits):
    return [int(bool(b)) for b in np.random.randint(2, size=num_bits)]

def bin_from_gray(g):
    b = [g[-1]]
    for n in g[-2::-1]:
        b.insert(0, b[0]^n)
    return b
    
def fenotipo(cromossomo, num_bits):
    b = bin_from_gray(cromossomo)
    u = sum(2**k for k in range(num_bits) if b[k])
    u /= 2**num_bits
    return -2 + 4 * u
    
def constroi_pop_rand(num_indv,num_bits):
    pop = []
    for i in range(num_indv):
        indv = init_indv_rand(num_bits)
        pop.append(indv)
        indv = []
    return np.array(pop)

def calc_fitness(populacao,num_indv,num_bits):
    fitness = []
    for i in range(num_indv):
        fitness.append(funcao_a_minimizar(fenotipo(list(populacao[i,:]),num_bits)))
    return fitness

num_bits = 16
num_indv = 5

# inicializa a populacao de forma aleatoria
populacao = constroi_pop_rand(num_indv,num_bits)

# avalia a populacao inicial
fitness = calc_fitness(populacao,num_indv,num_bits)
